**FRUIT RECOGNITION WITH CNN**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Conv2D,MaxPooling2D
from keras.layers import Dense, GlobalAveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import tensorflow as tf
from sklearn.datasets import load_files

# Input data files are available in the "../input/" directory.

import os
from os import listdir, makedirs
from os.path import join, exists, expanduser

# Any results you write to the current directory are saved as output.

In [ ]:
#Loading the file names and their respective target labels into an array

train_dir = '../input/fruits/fruits-360_dataset/fruits-360/Training/'
test_dir = '../input/fruits/fruits-360_dataset/fruits-360/Test/'

def load_dataset (path):
    data = load_files (path)
    files = np.array (data ['filenames'])
    targets = np.array (data ['target'])
    target_labels = np.array (data ['target_names'])
    return files, targets, target_labels

x_train, y_train, target_labels = load_dataset (train_dir)
x_test, y_test,_ = load_dataset (test_dir)
print ("LOADING COMPLETE")

print ("TRAINING SET SIZE: ", x_train.shape[0])
print ("TESTING SET SIZE: ", x_test.shape[0])
num_of_classes = len (np.unique (y_train))
print ("NUMBER OF CLASSES:", num_of_classes) #Confirming the number of classes

In [ ]:
img_width, img_height = 224, 224 # we set the img_width and img_height according to the pretrained models we are
# going to use. The input size for ResNet-50 is 224 by 224 by 3.

train_data_dir = '../input/fruits/fruits-360_dataset/fruits-360/Training/'
validation_data_dir = '../input/fruits/fruits-360_dataset/fruits-360/Test/'
nb_train_samples = 61488
nb_testing_samples = 20622
batch_size = 64

In [ ]:
print (y_train [0:10])

In [ ]:
from keras.utils import np_utils
y_train = np_utils.to_categorical (y_train, num_of_classes)
y_test = np_utils.to_categorical (y_test, num_of_classes)
y_train [0]

In [ ]:
#Dividing the validation set into test and validation set

x_test,x_valid = x_test[7000:],x_test[:7000]
y_test,y_vaild = y_test[7000:],y_test[:7000]
print('VALIDATION X : ',x_valid.shape)
print('VALIDATION Y :',y_vaild.shape)
print('TEST X : ',x_test.shape)
print('TEST Y : ',y_test.shape)

In [ ]:
x_train[0] #training and converting file names of images into pixel matrix

In [ ]:
#Loading file names in the set x and converting them into arrays

from keras.preprocessing.image import array_to_img, img_to_array, load_img

def convert_image_to_array(files):
    images_as_array=[]
    for file in files:
        # Convert to Numpy Array
        images_as_array.append(img_to_array(load_img(file)))
    return images_as_array

x_train = np.array(convert_image_to_array(x_train))
print('Training set shape : ',x_train.shape)

x_valid = np.array(convert_image_to_array(x_valid))
print('Validation set shape : ',x_valid.shape)

x_test = np.array(convert_image_to_array(x_test))
print('Test set shape : ',x_test.shape)

print('1st training image shape ',x_train[0].shape)

In [ ]:
print('1st training image as array',x_train[0])

In [ ]:
#Rescaling so all the pixel values lie within 0 - 1

x_train = x_train.astype('float32')/255
x_valid = x_valid.astype('float32')/255
x_test = x_test.astype('float32')/255
x_train[0]

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize =(30,5))
for i in range(10):
    ax = fig.add_subplot(2,5,i+1,xticks=[],yticks=[])
    ax.imshow(np.squeeze(x_train[i]))

In [ ]:
#Using three convolutional layers followed by maxpooling layers
#Lastly, we add dropout, flatten ans some fully connected layers (Dense)

model = Sequential()
model.add(Conv2D(filters = 16, kernel_size = (3,3), strides=(2,2),input_shape=(224,224,3),padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters = 32,kernel_size = 2,activation= 'relu',padding='same'))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters = 64,kernel_size = 2,activation= 'relu',padding='same'))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters = 128,kernel_size = 2,activation= 'relu',padding='same'))
model.add(MaxPooling2D(pool_size=2))

model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(200))
model.add(Activation('relu'))
model.add(Dropout(0.4))
model.add(Dense(120,activation = 'softmax'))
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
print('Compiled!')

In [ ]:
history_pretrained = model.fit_generator(train_generator,
                                         epochs=5, shuffle = True, verbose = 1, validation_data = validation_generator)

In [ ]:
#Loadung weights that yielded the best validation accuracy

model.load_weights('cnn_from_scratch_fruits.hdf5')

In [ ]:
#Evaluation

score = model.evaluate(x_test, y_test, verbose=0)
print('\n', 'Test accuracy:', score[1])

In [ ]:
#Test prediction visualisation

y_pred = model.predict(x_test)

# plot a random sample of test images, their predicted labels, and ground truth
fig = plt.figure(figsize=(16, 9))
for i, idx in enumerate(np.random.choice(x_test.shape[0], size=16, replace=False)):
    ax = fig.add_subplot(4, 4, i + 1, xticks=[], yticks=[])
    ax.imshow(np.squeeze(x_test[idx]))
    pred_idx = np.argmax(y_pred[idx])
    true_idx = np.argmax(y_test[idx])
    ax.set_title("{} ({})".format(target_labels[pred_idx], target_labels[true_idx]),
                 color=("green" if pred_idx == true_idx else "red"))

In [ ]:
plt.figure(1)   
plt.subplot(211)  
plt.plot(history.history['acc'])  
plt.plot(history.history['val_acc'])  
plt.title('model accuracy')  
plt.ylabel('accuracy')  
plt.xlabel('epoch')  
plt.legend(['train', 'test'], loc='upper left')   
   
plt.subplot(212)  
plt.plot(history.history['loss'])  
plt.plot(history.history['val_loss'])  
plt.title('model loss')  
plt.ylabel('loss')  
plt.xlabel('epoch')  
plt.legend(['train', 'test'], loc='upper left')  
plt.show()